In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "0"
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

import random
import torch
import torch.nn as NN
from torch.nn import functional as F
import matplotlib.pyplot as plt

with open('data/messages.txt', 'r', encoding='utf-8') as f:
    text = f.read()

N = 1000

In [2]:
import bpeasy
from bpeasy.tokenizer import BPEasyTokenizer

gpt4_regex = r"""(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"""
vocab = bpeasy.train_bpe(iter([text]), gpt4_regex, 10, N)
special_tokens = []
tokenizer = BPEasyTokenizer(vocab, gpt4_regex, [], fill_to_nearest_multiple_of_eight=True, name="skype_msgs_tok")

encode = lambda s: tokenizer.encode(s)
decode = lambda l: tokenizer.decode(l)
text_enc = encode(text)
LEN = len(text_enc)
CTX_SZ = 256 #8
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
sorted([v.decode('utf-8', 'replace') for v in vocab], key=lambda s: -len(s))

['???????????????',
 'beamingfacewith',
 '.washingtonpost',
 'moneymouthface',
 '(screamingfear',
 '??????????????',
 '=CantateDomino',
 'sweatgrinning',
 'screamingfear',
 '?????????????',
 '!!!!!!!!!!!!\n',
 '/megalophobia',
 '.theguardian',
 'fuckinglevel',
 '/interesting',
 'sinteresting',
 'antateDomino',
 '????????????',
 'gcIABCKBRBDM',
 '!!!!!!!!!!\n',
 'smilingeyes',
 'environment',
 '???????????',
 '=androidcss',
 'ingfacewith',
 '/Unexpected',
 '/kraftjrivo',
 'interesting',
 '-photograp',
 '!!!!!!!!!\n',
 'soundcloud',
 '(angryface',
 '=IanHubert',
 '-diffusion',
 'hysterical',
 'Projection',
 'handsinair',
 'sionsymbol',
 'kraftjrivo',
 '??????????',
 ' Diffusion',
 '/Damnthat',
 'ateDomino',
 '?????????',
 'nexpected',
 'diffusion',
 '-pictures',
 'screaming',
 'angryface',
 '(facepalm',
 'gcIABCKBR',
 'celebrate',
 '/comments',
 'mouthface',
 'rojection',
 '(zanyface',
 'idjourney',
 ' detailed',
 ' признал',
 ' заговор',
 ' ненавид',
 ' снаружи',
 'guardian',
 ' чуваков

In [4]:
xs, ys = [], []
for s in range(LEN - CTX_SZ):
    xs.append(text_enc[s:s+CTX_SZ])
    ys.append(text_enc[s+1:s+CTX_SZ+1])

tmp = list(zip(xs, ys))
random.shuffle(tmp)
xs, ys = zip(*tmp)
xs, ys = list(xs), list(ys)

n = int(0.9 * LEN)
xs_trn, ys_trn = torch.tensor(xs[:n], dtype=torch.int64), torch.tensor(ys[:n], dtype=torch.int64)
xs_val, ys_val = torch.tensor(xs[n:], dtype=torch.int64), torch.tensor(ys[n:], dtype=torch.int64)

xs_trn, ys_trn = xs_trn.to(device), ys_trn.to(device)
xs_val, ys_val = xs_val.to(device), ys_val.to(device)

In [3]:
class SelfAttentionHead(NN.Module):
    def __init__(self, head_sz, n_emb, ctx_sz, dropout):
        super().__init__()
        self.keys = NN.Linear(n_emb, head_sz, bias=False)
        self.queries = NN.Linear(n_emb, head_sz, bias=False)
        self.values = NN.Linear(n_emb, head_sz, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(ctx_sz, ctx_sz)))
        self.dropout = NN.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.keys(x) #    (B,T,C)
        q = self.queries(x) # (B,T,C)
        weights = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,T)
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        weights = F.softmax(weights, dim=-1) # (B,T,T)
        weights = self.dropout(weights)
        v = self.values(x) # (B,T,C)
        out = weights @ v # (B,T,C)
        return out

class SelfAttentionMultiHead(NN.Module):
    def __init__(self, n_heads, head_sz, n_emb, ctx_sz, dropout):
        super().__init__()
        self.heads = NN.ModuleList([SelfAttentionHead(head_sz, n_emb, ctx_sz, dropout) for _ in range(n_heads)])
        self.proj = NN.Linear(n_emb, n_emb)
        self.dropout = NN.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(NN.Module):
    def __init__(self, n_emb, multiplier, dropout):
        super().__init__()
        self.net = NN.Sequential(
            NN.Linear(n_emb, n_emb * multiplier),
            NN.ReLU(),
            NN.Linear(multiplier * n_emb, n_emb),
            NN.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class TransformerBlock(NN.Module):
    def __init__(self, n_heads, n_emb, ctx_sz, dropout):
        super().__init__()
        head_sz = n_emb // n_heads
        self.sa = SelfAttentionMultiHead(n_heads, head_sz, n_emb, ctx_sz, dropout)
        self.ffwd = FeedForward(n_emb, 4, dropout)
        self.ln1 = NN.LayerNorm(n_emb)
        self.ln2 = NN.LayerNorm(n_emb)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TransformerLanguageModel(NN.Module):
    def __init__(self, n_blocks, n_heads, voc_sz, n_emb, ctx_sz, dropout):
        super().__init__()
        self.ctx_sz = ctx_sz
        self.tok_emb_table = NN.Embedding(voc_sz, n_emb)
        self.pos_emb_table = NN.Embedding(ctx_sz, n_emb)
        self.blocks = NN.Sequential(*(
            ([TransformerBlock(n_heads, n_emb, ctx_sz, dropout)] * n_blocks) +
            [NN.LayerNorm(n_emb)]
        ))
        self.lm_head = NN.Linear(n_emb, voc_sz)
    def forward(self, idx, targets=None):
        B, T = idx.shape        
        tok_emb = self.tok_emb_table(idx)
        pos_emb = self.pos_emb_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            loss = F.cross_entropy(logits, targets.view(B * T))        
        return logits, loss
    def generate(self, idx, max_new_tok):
        for _ in range(max_new_tok):
            idx_crop = idx[:, -self.ctx_sz:]
            logits, _ = self(idx_crop)
            probs = F.softmax(logits[:, -1, :], dim=1)
            idx_nxt = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_nxt), dim=1)
        return idx

In [4]:
N_EMB = 384 #32
N_HEADS = 6
N_BLOCKS = 6
EVAL_INT = 10
model = TransformerLanguageModel(N_BLOCKS, N_HEADS, N, N_EMB, CTX_SZ, 0.2).to(device)

In [5]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

@torch.no_grad()
def est_loss(model, d):
    out = {}
    model.eval()
    for splt in ['trn', 'val']:
        lossi = torch.zeros(EVAL_INT)
        for k in range(EVAL_INT):
            bix = torch.randint(d[splt][0].shape[0] - 1, (BATCH_SZ,)).to(device)
            _, loss = model(d[splt][0][bix], d[splt][1][bix])
            lossi[k] = loss.item()
        out[splt] = lossi.mean()
    model.train()
    return out

BATCH_SZ = 96 #32
#lossi = {'trn':[], 'val':[]}
#for i in range(3500):
#    xb, yb = xs_trn[i*BATCH_SZ : (i+1)*BATCH_SZ], ys_trn[i*BATCH_SZ : (i+1)*BATCH_SZ]
#    logits, loss = model(xb, yb)
#    optimizer.zero_grad(set_to_none=True)
#    loss.backward()
#    optimizer.step()
#    print(i,end=',')
#    if i % 500 == 0:
#        est = est_loss(model, {'trn': [xs_trn, ys_trn], 'val': [xs_val, ys_val]})
#        lossi['trn'].append(est['trn'])
#        lossi['val'].append(est['val'])
#        print(est['trn'])
#plt.plot(lossi['trn'])
#plt.plot(lossi['val'])

<All keys matched successfully>

In [5]:
model.load_state_dict(torch.load('./baby.m'))

<All keys matched successfully>

In [7]:
import gc
with torch.no_grad():
    model = None
    gc.collect()
    torch.cuda.empty_cache()

In [11]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 11.576MB


In [11]:
xs_trn.shape

torch.Size([480148, 256])

In [7]:
import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)
len(text)
device

2641384


'cuda'

In [6]:
print(decode(model.generate(torch.zeros((1,1), dtype=torch.long).to(device), max_new_tok=1000)[0].tolist())) 

 асали ней не приостанет, и уви
ТИМА:	.. пока ничего!!
	не и я не провера речениием человекить? (ко вря)
ВАНЯ:	Вота лет зналогfacepalml) 
ВАСЯ:	это вы меня не поменьглямащьях, если то что это былитьсячика
	постскуюты Кинское
ВАСЯ:	ного крови...
ТИМА:	<ЦИТАТА:тва четниковить про нейроски и не понятельные роды
ВАНЯ:	я споко много остров, того изба реаклаюств, чего разум
ТИМА:	можал) я нашёл вычала Ева сделали - сидимпи изобря
ВАНЯ:	у-за
ТИМА:	хорош режется
ВАСЯ:	ну этоильу
ВАНЯ:	
МАША:	Дейец латил люе стать же закой. Матем
ТИМА:	я нашёлся, чемчёл рыоррать-ли)
	и датычами
	с что не проверу
ВАНЯ:	и там этася
ТИМА:	 Чинтер мо, и очень мо мил обажение�асным не дом м Катоему 7 нет сибит
ВАНЯ:	+-ли
ВАСЯ:	<ЦИТАТА:
	яида воскловные смыбуществу-нему:	а мечатекрывые в уже не вернение)е станнеи!
	я уже:	это час)
ВАНЯ:	 судляет двух всё радик
ТИМА:	делыйтися
СТЁПА:	а бига всё никакмо нос восскующий...
ВАСЯ: опередуточше пока
	а о Голиняковок нал)
ТИМА:	нехоили!
ВАНЯ:	Bдя в кр взачестве хватка?
	В кл

In [9]:
print(decode(model.generate(torch.tensor([encode("ВАНЯ:	какой-то у них экран растянутый")], dtype=torch.long).to(device), max_new_tok=1000)[0].tolist())) 

ВАНЯ:	какой-то у них экран растянутый
	слишком сильно натянули
ТИМА:	лишь бы не лопнул во время фильма!
	я вот ещё боюсь, что Гриша, Стас, Зина и Евгений Петрович как всегда будут загораживать нам и без того низкий экран
ВАНЯ:	очень может быть
	берите Фридриха, он на них повлияет
	правда, солидит
	правда, сам он тот, кто обкался
ТИМА:	он ещё крендель голяк
СТЁПА:	У нее было конечно!
ТИМА:	<КАРТИНКА>
	что ты написал -? Кел?
ТИМА:	это где?
ВАНЯ:	<КАРТИНКА>
ТИМА:	ну он знает, не, прикрывал
	это что ты хочешь?)
	слушайду, не сегодня
СТЁПА:	<КАРТИНКА>
	<ЧТО-ТО>
	<КАРТИНКА>
ТИМА:	это видимо ИИ от Провали
	другками, так всё это
СТЁПА:	ну вот так, когдаобраз будет жить хочет
ТИМА:	<ССЫЛКА:https://www.youtube.com/watch?v=LexX, и да
ВАНЯ:	<ССЫЛКА:https://www.youtube.com/watch?v=7SxBXs>
	<ССЫЛКА:https://www.youtube.com/watch?v=4OKZPhM&ab_channel=SulfjosemperAFwo3WACAiJ2G7A16>
2024-09-1T
	<КАРТИНКА>
2024-06T
СТЁПА:	Красиво!
	Это не могу вспомнить делитесь, он их чувак делать?
ТИМА:	причиталежал на